In [ ]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import time
import cv2
from sys import getsizeof
from IPython.display import display
import random
import math
import datetime
import sys

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import *
#tf.config.optimizer.set_jit(True)
#tf.debugging.set_log_device_placement(True)
print(tf.config.experimental.list_physical_devices())

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
N_EPOCHS = 20
TRAIN_P = 0.8
VAL_P = 0.2
TEST_P = 0


%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb

id_map = get_selected_taxons("../../selected_taxons.txt")
n_classes = len(list(id_map.keys()))

# Handle data

In [ ]:
files, labels = get_dataset()
NB_SAMPLES = len(files)

In [ ]:
#for image, label in full_dataset.take(1):
#    print(image.numpy().shape)
#    print(label.numpy())
#    display(Image.fromarray(np.uint8(image.numpy()*255)))

In [ ]:
def cb_load_image(image_path, label):
    img_file = tf.io.read_file(image_path)
    img = tf.image.decode_png(img_file, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32) #0-1 range
    return img, label

full_dataset = tf.data.Dataset.from_tensor_slices((files, tf.keras.utils.to_categorical(labels, num_classes=None, dtype='float32')))
full_dataset = full_dataset.shuffle(len(files))
full_dataset = full_dataset.map(cb_load_image, num_parallel_calls=AUTOTUNE)

##full_dataset = full_dataset.prefetch(1)

In [ ]:
DATASET_SIZE = len(files)
train_size = int(TRAIN_P * DATASET_SIZE)
val_size = int(VAL_P * DATASET_SIZE)
test_size = int(TEST_P * DATASET_SIZE)

train_dataset = full_dataset.take(train_size)
tmp_dataset = full_dataset.skip(train_size)
val_dataset = tmp_dataset.take(val_size)
tmp_dataset = full_dataset.skip(val_size)
test_dataset = tmp_dataset.take(test_size)

print(tf.data.experimental.cardinality(train_dataset).numpy())
print(tf.data.experimental.cardinality(val_dataset).numpy())

# Model desgin

In [ ]:
# fetching base model
#base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling=None)
input_tensor = Input(shape=(256, 256, 3))
base_model = InceptionV3(weights='imagenet', input_tensor=input_tensor, include_top=False)

In [ ]:
# setting model for specifiv case
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(230, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Training

In [ ]:
# Setting tensorboard
!rm -rf LOG_DIR
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## New layers only

In [ ]:
print(int(0.1*N_EPOCHS), "epochs composed of", (int(train_size/BATCH_SIZE)-1), "batches (steps) of", BATCH_SIZE, "images.")

In [ ]:
n_epochs_train1 = 3

train_train1 = train_dataset.repeat(n_epochs_train1).batch(BATCH_SIZE)
val_train1 = val_dataset.repeat(n_epochs_train1).batch(BATCH_SIZE)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_train1, 
          epochs=n_epochs_train1, 
          steps_per_epoch=int(train_size/BATCH_SIZE),
          use_multiprocessing=True, 
          validation_data=val_train1,
          validation_steps=int(val_size/BATCH_SIZE),
          callbacks=[tensorboard_callback],
          initial_epoch=0)

In [ ]:
train_train1 = train_dataset.repeat(n_epochs_train1).batch(BATCH_SIZE)
val_train1 = val_dataset.repeat(n_epochs_train1).batch(BATCH_SIZE)

print(n_epochs_train1*int(train_size/BATCH_SIZE))
print(tf.data.experimental.cardinality(train_train1).numpy())
print(n_epochs_train1*int(val_size/BATCH_SIZE))
print(int(tf.data.experimental.cardinality(val_train1).numpy()))
val_size

## Training 2 last inceptions blocks

In [ ]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

In [ ]:
n_epochs_train2 = n_epochs_train1+10

train_train2 = train_dataset.repeat(n_epochs_train2).batch(BATCH_SIZE)
val_train2 = val_dataset.repeat(n_epochs_train2).batch(BATCH_SIZE)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True


model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_train2, 
          epochs=n_epochs_train2, 
          steps_per_epoch=int(train_size/BATCH_SIZE)-1,
          use_multiprocessing=True, 
          validation_data=val_train2,
          validation_steps=int(val_size/BATCH_SIZE)-1,
          callbacks=[tensorboard_callback],
          initial_epoch=n_epochs_train1)

## Training full model

In [ ]:
n_epochs_train3 = n_epochs_train2+20

train_train3 = train_dataset.repeat(n_epochs_train3).batch(BATCH_SIZE)
val_train3 = val_dataset.repeat(n_epochs_train3).batch(BATCH_SIZE)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_train3, 
          epochs=n_epochs_train3, 
          steps_per_epoch=int(train_size/BATCH_SIZE)-1,
          use_multiprocessing=True, 
          validation_data=val_train3,
          validation_steps=int(val_size/BATCH_SIZE)-1,
          callbacks=[tensorboard_callback],
          initial_epoch=n_epochs_train2)

## Saving model

In [ ]:
save_model(model, SAVED_MODELS_ROOT)